In [6]:
import os
import numpy as np
import time
import tensorflow as tf
import datetime
import matplotlib.pyplot as plt
seed = 5691
np.random.seed(seed)
tf.random.set_seed(seed)
NUM_OF_CATEGORY = 10

In [7]:
mnist = tf.keras.datasets.fashion_mnist
# mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = np.reshape(x_train / 255.0, (int(tf.shape(x_train)[0]),-1)), np.reshape(x_test / 255.0, (int(tf.shape(x_test)[0]),-1))
one_hot_y = lambda t: [1 if i == t else 0 for i in range(NUM_OF_CATEGORY)]
y_train, y_test = np.array([one_hot_y(y) for y in y_train]), np.array([one_hot_y(y) for y in y_test])


In [8]:
class MLP(tf.keras.Model):
    def __init__(self, size_input, size_hidden, size_output, device=None):
        super(MLP, self).__init__()
        """
        size_input: int, size of input layer
        size_hidden: int, size of hidden layer
        size_output: int, size of output layer
        device: str or None, either 'cpu' or 'gpu' or None. If None, the device to be used will be decided automatically during Eager Execution
        """
        self.size_input, self.size_hidden, self.size_output, self.device =\
        size_input, size_hidden, size_output, device

        # Initialize weights between input layer and hidden layer 1
        self.W1 = tf.Variable(tf.random.normal([self.size_input, self.size_hidden], seed=seed))
        # Initialize biases for hidden layer 1
        self.b1 = tf.Variable(tf.random.normal([1, self.size_hidden], seed=seed))
         # Initialize weights between hidden layer 1 and hidden layer 2
        self.W2 = tf.Variable(tf.random.normal([self.size_hidden, self.size_hidden], seed=seed))
        # Initialize biases for hidden layer 2
        self.b2 = tf.Variable(tf.random.normal([1, self.size_hidden], seed=seed))
         # Initialize weights between hidden layer 2 and output layer
        self.W3 = tf.Variable(tf.random.normal([self.size_hidden, self.size_output], seed=seed))
        # Initialize biases for output layer
        self.b3 = tf.Variable(tf.random.normal([1, self.size_output], seed=seed))

        # Define variables to be updated during backpropagation
        self.MLP_variables = [self.W1, self.W2, self.W3, self.b1, self.b2, self.b3]
        # self.MLP_variables = [self.W1, self.W3, self.b1, self.b3]

    
    def forward(self, X, dropout_rate=0):
        """
        forward pass
        X: Tensor, inputs
        """
        if self.device is not None:
            with tf.device('gpu:0' if self.device=='gpu' else 'cpu'):
                self.y = self.compute_output(X, dropout_rate=dropout_rate)
        else:
            self.y = self.compute_output(X, dropout_rate=dropout_rate)

        return self.y
  
    def loss(self, y_pred, y_true, L1=0, L2=0):
        '''
        y_pred - Tensor of shape (batch_size, size_output)
        y_true - Tensor of shape (batch_size, size_output)
        '''
        y_true_tf = tf.cast(tf.reshape(y_true, (-1, self.size_output)), dtype=tf.float32)
        y_pred_tf = tf.cast(y_pred, dtype=tf.float32)
        l2_penlty = tf.nn.l2_loss(self.W1)*L2
        loss_with_l2 = l2_penlty+tf.losses.categorical_crossentropy(y_true_tf, y_pred_tf)
        return loss_with_l2
  
    def backward(self, X_train, y_train, dropout_rate, learning_rate, L1=0, L2=0):
        """
        backward pass
        """
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
        with tf.GradientTape() as tape:
            predicted = self.forward(X_train,dropout_rate=dropout_rate)
            current_loss = self.loss(predicted, y_train)
        grads = tape.gradient(current_loss, self.MLP_variables)
        optimizer.apply_gradients(zip(grads, self.MLP_variables))
        return current_loss, predicted
        
        
    def compute_output(self, X, dropout_rate=0):
        """
        Custom method to obtain output tensor during forward pass
        """
        # Cast X to float32
        X_tf = tf.cast(X, dtype=tf.float32)
        #Remember to normalize your dataset before moving forward
        # Compute values in hidden layer 1
        what = tf.matmul(X_tf, self.W1) + self.b1
        hhat = tf.nn.relu(what)
        # Implement Dropout
        hhat = tf.nn.dropout(hhat, rate = dropout_rate, seed=seed)

        # Compute values in hidden layer 2
        what = tf.matmul(hhat, self.W2) + self.b2
        hhat = tf.nn.relu(what)
        # Implement Dropout
        hhat = tf.nn.dropout(hhat, rate = dropout_rate, seed=seed)

        # Compute output
        output = tf.matmul(hhat, self.W3) + self.b3
        #Now consider two things , First look at inbuild loss functions if they work with softmax or not and then change this
        #Second add tf.Softmax(output) and then return this variable
        return tf.nn.softmax(output)
#         return output


In [9]:
loss_for_ten = []
for s in range(10):
    print(s,"!!!!!!!!!!!!!!!!!")
    DROPOUT_RATE = 0
    BATCH_SIZE = 200
    HIDDEN_SIZE = 256
    NUM_EPOCHS = 150
    LEARNING_RATE = 0.003
    L2_PENLTY = 0

    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    path = f"SEED_{s}-LearnRate_{LEARNING_RATE}-L2_{L2_PENLTY}-DROPOUT_{DROPOUT_RATE}-BATCH_{BATCH_SIZE}-HIDDEN_{HIDDEN_SIZE}-TIME_{current_time}"

    # Set log summary

    train_log_dir = 'logs/mnist_fashion/' + path + '/train'
    test_log_dir = 'logs/mnist_fashion/' + path + '/test'
#     train_log_dir = 'logs/mnist/' + path + '/train'
#     test_log_dir = 'logs/mnist/' + path + '/test'
    train_summary_writer = tf.summary.create_file_writer(train_log_dir)
    test_summary_writer = tf.summary.create_file_writer(test_log_dir)


    size_input = int(tf.shape(x_train)[1])
    size_hidden = HIDDEN_SIZE
    size_output = int(tf.shape(y_train)[1])
    number_of_train_examples = int(tf.shape(x_train)[0])
    number_of_test_examples = int(tf.shape(x_test)[0])


    # print("size_input",size_input)
    # print("size_output",size_output)
    # print("number_of_train_examples",number_of_train_examples)
    # print("number_of_test_examples",number_of_test_examples)


    mlp_on_gpu = MLP(size_input, size_hidden, size_output, device='cpu')
    time_start = time.time()
    epoch = 1
    loss_diff,last_loss = 1,0

    while epoch <= NUM_EPOCHS and abs(loss_diff) > 0.00001:
        loss_total_gpu = tf.zeros([1,1], dtype=tf.float32)
        train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(BATCH_SIZE+BATCH_SIZE//4, seed=epoch*(seed)).batch(BATCH_SIZE)
        for inputs, outputs in train_ds:
            cur_loss, preds = mlp_on_gpu.backward(inputs, outputs, dropout_rate=DROPOUT_RATE, learning_rate=LEARNING_RATE, L2=L2_PENLTY)
            loss_total_gpu += cur_loss
      # Calculate Accuracy
        train_accuracy, test_accuracy = tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.CategoricalAccuracy()
        train_accuracy.update_state(y_train, mlp_on_gpu.forward(x_train))
        test_accuracy.update_state(y_test, mlp_on_gpu.forward(x_test))
        train_loss = np.sum(loss_total_gpu) / x_train.shape[0]
        test_loss = np.sum(mlp_on_gpu.loss(mlp_on_gpu.forward(x_test), y_test)) / x_test.shape[0]

        with train_summary_writer.as_default():
            tf.summary.scalar('loss', train_loss, step=epoch)
            tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

        with test_summary_writer.as_default():
            tf.summary.scalar('loss', test_loss, step=epoch)
            tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

        loss_diff = train_loss - last_loss
        last_loss = train_loss
        print(f'Number of Epoch = {epoch} - Training Cross Entropy:= {np.sum(loss_total_gpu) / x_train.shape[0]} - Training Accuracy:= {train_accuracy.result().numpy()} - Test Accuracy:= {test_accuracy.result().numpy()}')
        time_taken = time.time() - time_start
        print('Time taken (in seconds): {:.2f}'.format(time_taken))
        time_start = time.time()
        epoch += 1
    loss_for_ten.append(last_loss)


0 !!!!!!!!!!!!!!!!!
Number of Epoch = 1 - Training Cross Entropy:= 13.760577083333333 - Training Accuracy:= 0.26053333282470703 - Test Accuracy:= 0.2563000023365021
Time taken (in seconds): 6.34
Number of Epoch = 2 - Training Cross Entropy:= 12.018980208333334 - Training Accuracy:= 0.27258333563804626 - Test Accuracy:= 0.2703000009059906
Time taken (in seconds): 6.18
Number of Epoch = 3 - Training Cross Entropy:= 11.641939583333333 - Training Accuracy:= 0.2793833315372467 - Test Accuracy:= 0.2771000266075134
Time taken (in seconds): 5.88
Number of Epoch = 4 - Training Cross Entropy:= 11.593435416666667 - Training Accuracy:= 0.27968332171440125 - Test Accuracy:= 0.2766000032424927
Time taken (in seconds): 6.09
Number of Epoch = 5 - Training Cross Entropy:= 11.554922916666667 - Training Accuracy:= 0.2858666777610779 - Test Accuracy:= 0.2833000123500824
Time taken (in seconds): 6.14
Number of Epoch = 6 - Training Cross Entropy:= 11.573202083333333 - Training Accuracy:= 0.27994999289512634

Number of Epoch = 49 - Training Cross Entropy:= 8.316402083333333 - Training Accuracy:= 0.4864666759967804 - Test Accuracy:= 0.48350003361701965
Time taken (in seconds): 6.17
Number of Epoch = 50 - Training Cross Entropy:= 8.329055208333333 - Training Accuracy:= 0.48208335041999817 - Test Accuracy:= 0.4790000319480896
Time taken (in seconds): 5.91
Number of Epoch = 51 - Training Cross Entropy:= 8.303678125 - Training Accuracy:= 0.48643332719802856 - Test Accuracy:= 0.4837000370025635
Time taken (in seconds): 6.28
Number of Epoch = 52 - Training Cross Entropy:= 8.285264583333333 - Training Accuracy:= 0.4854666590690613 - Test Accuracy:= 0.482200026512146
Time taken (in seconds): 6.24
Number of Epoch = 53 - Training Cross Entropy:= 8.310090625 - Training Accuracy:= 0.48678332567214966 - Test Accuracy:= 0.48410001397132874
Time taken (in seconds): 6.21
Number of Epoch = 54 - Training Cross Entropy:= 8.301053125 - Training Accuracy:= 0.4812166690826416 - Test Accuracy:= 0.4783000349998474


Number of Epoch = 97 - Training Cross Entropy:= 8.274904166666667 - Training Accuracy:= 0.4880666732788086 - Test Accuracy:= 0.4848000109195709
Time taken (in seconds): 6.08
Number of Epoch = 98 - Training Cross Entropy:= 8.268926041666667 - Training Accuracy:= 0.48721668124198914 - Test Accuracy:= 0.4846000373363495
Time taken (in seconds): 6.00
Number of Epoch = 99 - Training Cross Entropy:= 8.278890625 - Training Accuracy:= 0.48669999837875366 - Test Accuracy:= 0.48420003056526184
Time taken (in seconds): 6.17
Number of Epoch = 100 - Training Cross Entropy:= 8.284980729166668 - Training Accuracy:= 0.486133337020874 - Test Accuracy:= 0.48420003056526184
Time taken (in seconds): 6.05
Number of Epoch = 101 - Training Cross Entropy:= 8.286644791666667 - Training Accuracy:= 0.48668333888053894 - Test Accuracy:= 0.484000027179718
Time taken (in seconds): 6.14
Number of Epoch = 102 - Training Cross Entropy:= 8.298685416666666 - Training Accuracy:= 0.4881666600704193 - Test Accuracy:= 0.484

Number of Epoch = 145 - Training Cross Entropy:= 8.256291145833334 - Training Accuracy:= 0.48891666531562805 - Test Accuracy:= 0.4854000210762024
Time taken (in seconds): 6.21
Number of Epoch = 146 - Training Cross Entropy:= 8.241434375 - Training Accuracy:= 0.48899999260902405 - Test Accuracy:= 0.48600003123283386
Time taken (in seconds): 6.23
Number of Epoch = 147 - Training Cross Entropy:= 8.247383333333334 - Training Accuracy:= 0.48848333954811096 - Test Accuracy:= 0.48520001769065857
Time taken (in seconds): 6.20
Number of Epoch = 148 - Training Cross Entropy:= 8.263105729166666 - Training Accuracy:= 0.48311668634414673 - Test Accuracy:= 0.4805000126361847
Time taken (in seconds): 6.21
Number of Epoch = 149 - Training Cross Entropy:= 8.249452604166667 - Training Accuracy:= 0.48901668190956116 - Test Accuracy:= 0.48590001463890076
Time taken (in seconds): 6.21
Number of Epoch = 150 - Training Cross Entropy:= 8.284896354166667 - Training Accuracy:= 0.48680001497268677 - Test Accurac

Number of Epoch = 43 - Training Cross Entropy:= 9.517302083333334 - Training Accuracy:= 0.4138833284378052 - Test Accuracy:= 0.40880000591278076
Time taken (in seconds): 6.19
Number of Epoch = 44 - Training Cross Entropy:= 9.498725 - Training Accuracy:= 0.3950500190258026 - Test Accuracy:= 0.387800008058548
Time taken (in seconds): 6.22
Number of Epoch = 45 - Training Cross Entropy:= 9.6138625 - Training Accuracy:= 0.4060666859149933 - Test Accuracy:= 0.39810001850128174
Time taken (in seconds): 6.20
Number of Epoch = 46 - Training Cross Entropy:= 9.539252083333333 - Training Accuracy:= 0.4097000062465668 - Test Accuracy:= 0.4044000208377838
Time taken (in seconds): 5.89
Number of Epoch = 47 - Training Cross Entropy:= 9.533882291666666 - Training Accuracy:= 0.40950000286102295 - Test Accuracy:= 0.4037000238895416
Time taken (in seconds): 6.23
Number of Epoch = 48 - Training Cross Entropy:= 9.497588541666667 - Training Accuracy:= 0.4073166847229004 - Test Accuracy:= 0.4004000127315521
T

...

Number of Epoch = 88 - Training Cross Entropy:= 10.171460416666667 - Training Accuracy:= 0.37123334407806396 - Test Accuracy:= 0.3700000047683716
Time taken (in seconds): 6.38
Number of Epoch = 89 - Training Cross Entropy:= 10.080121875 - Training Accuracy:= 0.3706333339214325 - Test Accuracy:= 0.3684000074863434
Time taken (in seconds): 6.18
Number of Epoch = 90 - Training Cross Entropy:= 10.084280208333332 - Training Accuracy:= 0.3778499960899353 - Test Accuracy:= 0.3735000193119049
Time taken (in seconds): 6.23
Number of Epoch = 91 - Training Cross Entropy:= 10.034314583333334 - Training Accuracy:= 0.37763333320617676 - Test Accuracy:= 0.3741000294685364
Time taken (in seconds): 5.83
Number of Epoch = 92 - Training Cross Entropy:= 10.05005625 - Training Accuracy:= 0.37646666169166565 - Test Accuracy:= 0.3742000162601471
Time taken (in seconds): 6.22
Number of Epoch = 93 - Training Cross Entropy:= 10.055432291666667 - Training Accuracy:= 0.3780166804790497 - Test Accuracy:= 0.3749000

Number of Epoch = 135 - Training Cross Entropy:= 10.057214583333334 - Training Accuracy:= 0.37665000557899475 - Test Accuracy:= 0.3725000321865082
Time taken (in seconds): 6.17
Number of Epoch = 136 - Training Cross Entropy:= 10.038740625 - Training Accuracy:= 0.3783833384513855 - Test Accuracy:= 0.37460002303123474
Time taken (in seconds): 6.16
Number of Epoch = 137 - Training Cross Entropy:= 10.029357291666667 - Training Accuracy:= 0.3782833516597748 - Test Accuracy:= 0.37540000677108765
Time taken (in seconds): 5.92
Number of Epoch = 138 - Training Cross Entropy:= 10.019404166666666 - Training Accuracy:= 0.37826666235923767 - Test Accuracy:= 0.3750000298023224
Time taken (in seconds): 6.23
Number of Epoch = 139 - Training Cross Entropy:= 10.022155208333333 - Training Accuracy:= 0.3777666687965393 - Test Accuracy:= 0.3765000104904175
Time taken (in seconds): 6.19
Number of Epoch = 140 - Training Cross Entropy:= 10.087575 - Training Accuracy:= 0.37746667861938477 - Test Accuracy:= 0.3

Number of Epoch = 33 - Training Cross Entropy:= 8.093496875 - Training Accuracy:= 0.4915333390235901 - Test Accuracy:= 0.4861000180244446
Time taken (in seconds): 6.17
Number of Epoch = 34 - Training Cross Entropy:= 8.189367708333334 - Training Accuracy:= 0.4942333400249481 - Test Accuracy:= 0.4812000095844269
Time taken (in seconds): 5.87
Number of Epoch = 35 - Training Cross Entropy:= 8.214921354166666 - Training Accuracy:= 0.49588334560394287 - Test Accuracy:= 0.48730000853538513
Time taken (in seconds): 6.20
Number of Epoch = 36 - Training Cross Entropy:= 8.126209895833334 - Training Accuracy:= 0.4953833520412445 - Test Accuracy:= 0.4847000241279602
Time taken (in seconds): 6.16
Number of Epoch = 37 - Training Cross Entropy:= 8.106033333333333 - Training Accuracy:= 0.49959999322891235 - Test Accuracy:= 0.4886000156402588
Time taken (in seconds): 6.17
Number of Epoch = 38 - Training Cross Entropy:= 8.09813125 - Training Accuracy:= 0.48286667466163635 - Test Accuracy:= 0.474700033664

Number of Epoch = 81 - Training Cross Entropy:= 7.557244270833333 - Training Accuracy:= 0.540066659450531 - Test Accuracy:= 0.5235000252723694
Time taken (in seconds): 5.90
Number of Epoch = 82 - Training Cross Entropy:= 7.521690625 - Training Accuracy:= 0.5364166498184204 - Test Accuracy:= 0.5236999988555908
Time taken (in seconds): 6.22
Number of Epoch = 83 - Training Cross Entropy:= 7.376941666666666 - Training Accuracy:= 0.5271000266075134 - Test Accuracy:= 0.5164999961853027
Time taken (in seconds): 6.31
Number of Epoch = 84 - Training Cross Entropy:= 7.521129166666666 - Training Accuracy:= 0.5489833354949951 - Test Accuracy:= 0.5365000367164612
Time taken (in seconds): 6.19
Number of Epoch = 85 - Training Cross Entropy:= 7.329865625 - Training Accuracy:= 0.5549166798591614 - Test Accuracy:= 0.5418000221252441
Time taken (in seconds): 6.23
Number of Epoch = 86 - Training Cross Entropy:= 7.270730208333333 - Training Accuracy:= 0.5390000343322754 - Test Accuracy:= 0.5285000205039978

Number of Epoch = 129 - Training Cross Entropy:= 7.230545833333333 - Training Accuracy:= 0.5526999831199646 - Test Accuracy:= 0.5415000319480896
Time taken (in seconds): 6.52
Number of Epoch = 130 - Training Cross Entropy:= 7.2666546875 - Training Accuracy:= 0.5565500259399414 - Test Accuracy:= 0.5484000444412231
Time taken (in seconds): 6.20
Number of Epoch = 131 - Training Cross Entropy:= 7.272133333333334 - Training Accuracy:= 0.5625 - Test Accuracy:= 0.551800012588501
Time taken (in seconds): 6.21
Number of Epoch = 132 - Training Cross Entropy:= 7.183500520833333 - Training Accuracy:= 0.5568333268165588 - Test Accuracy:= 0.5437000393867493
Time taken (in seconds): 6.12
Number of Epoch = 133 - Training Cross Entropy:= 7.324544791666667 - Training Accuracy:= 0.5622833371162415 - Test Accuracy:= 0.5516000390052795
Time taken (in seconds): 6.15
Number of Epoch = 134 - Training Cross Entropy:= 7.1909875 - Training Accuracy:= 0.5586000084877014 - Test Accuracy:= 0.5511000156402588
Time t

In [10]:
loss_for_ten

[8.284896354166667,
 9.348185416666666,
 7.466494791666666,
 5.440867708333333,
 6.876169791666666,
 7.5873390625,
 7.510666666666666,
 12.9189375,
 10.029638541666667,
 7.114257291666667]